0. Download sequence and structure files from ZINC-downloader-3D-{smi}/{sdf.gz}.uri

In [ ]:
# run the following in the terminal (remove the "!" at the first)
!wget -i ZINC-downloader-3D-sdf.gz.uri -P ./sdf
!wget -i ZINC-downloader-3D-smi.uri -P ./smile

1. Extract SMILES sequence from .smi format files

In [ ]:
import os
import csv

input_folder = "./smile"
output_file = "./zinc_SMILES.csv"
all_data = []

for filename in os.listdir(input_folder):
    if filename.endswith(".smi"):
        file_path = os.path.join(input_folder, filename)
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines[0:]:
                parts = line.strip().split()
                if len(parts) >= 2:
                    smiles = parts[0]
                    zinc_id = parts[1]
                    all_data.append([zinc_id, smiles])

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['zinc_id', 'smiles'])
    writer.writerows(all_data)

print(f"数据已成功整合到 {output_file}")

2. Unzip .sdf.gz files

In [ ]:
import gzip
import shutil
import os

# 设置文件夹路径（替换为你的路径）
folder_path = "./sdf"

for filename in os.listdir(folder_path):
    if filename.endswith(".sdf.gz"):
        # 构建完整文件路径
        gz_path = os.path.join(folder_path, filename)
        # 生成解压后的文件名（去掉.gz后缀）
        output_name = filename[:-3]  # 假设文件名类似 BAAAMO.xaa.sdf.gz
        output_path = os.path.join('./structure', output_name)

        # 执行解压
        with gzip.open(gz_path, 'rb') as f_in:
            with open(output_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"解压完成: {filename} → {output_name}")

print("所有文件已处理完毕！")


3. Split sdf files (This is necessary, because the sdf files contain many molecules in a file)

In [ ]:
import os

def split_sdf(input_folder, output_folder):
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)

    # 遍历输入文件夹中的所有文件
    for filename in os.listdir(input_folder):
        if filename.endswith(".sdf"):
            input_path = os.path.join(input_folder, filename)

            with open(input_path, 'r') as f:
                current_molecule = []
                zinc_id = None

                for line in f:
                    current_molecule.append(line)

                    # 检测分子开始
                    if line.startswith("ZINC"):
                        zinc_id = line.strip().split()[0]  # 提取ZINC ID

                    # 检测分子结束
                    if line.strip() == "$$$$":
                        # 写入文件
                        output_path = os.path.join(output_folder, f"{zinc_id}.sdf")
                        with open(output_path, 'w') as out_file:
                            out_file.writelines(current_molecule)

                        # 重置临时存储
                        current_molecule = []
                        zinc_id = None
# 使用示例
split_sdf("./structure", "./zinc_sdf")

4. Construct input molecule file (in a format of .pkl.gz)

In [ ]:
# run "python preprocess_mol_pkl.py" in the ./zinc file

5. Make sure SMILES sequence and molecule structure have the same ID

In [ ]:
import csv
import gzip
import pickle

# 定义文件路径
pkl_file_path = "../mol_structures.pkl.gz"
csv_file_path = "zinc_SMILES.csv"
output_file_path = "filtered_zinc_SMILES.csv"

# 获取 JSON 文件中的所有 ID
def load_dict(input_path):
    with gzip.open(input_path, 'rb') as f:
        return pickle.load(f)

data = load_dict(pkl_file_path)
ids = set(data.keys())
print(f"pkl 文件中不同 ID 的数量: {len(ids)}")

# 筛选 CSV 文件中与 JSON 文件具有相同 ID 的项
with open(csv_file_path, 'r') as csv_file, open(output_file_path, 'w', newline='') as output_file:
    csv_reader = csv.DictReader(csv_file)
    csv_writer = csv.DictWriter(output_file, fieldnames=csv_reader.fieldnames)
    csv_writer.writeheader()

    matched_ids = set()
    for row in csv_reader:
        zinc_id = row.get('zinc_id')
        if zinc_id and zinc_id in ids:
            csv_writer.writerow(row)
            matched_ids.add(zinc_id)

    print(f"CSV 文件中匹配的 ID 数量: {len(matched_ids)}")
    print(f"筛选后的 CSV 文件已保存到 {output_file_path}")

# 验证两者所包含的 ID 是否一致
print(f"pkl 文件中的 ID 数量: {len(ids)}")
print(f"筛选后的 CSV 文件中的 ID 数量: {len(matched_ids)}")
print(f"两者所包含的 ID 是否一致: {len(ids) == len(matched_ids)}")

6. Extract protein pocket information

In [ ]:
# # Batch acquisition of protein sequence, pocket sequence, and absolute position information of pockets
import pandas as pd
def get_poc_seq(pocket_path, protein_path):
    aa_codes = {
        'ALA':'A', 'CYS':'C', 'ASP':'D', 'GLU':'E',
        'PHE':'F', 'GLY':'G', 'HIS':'H', 'LYS':'K',
        'ILE':'I', 'LEU':'L', 'MET':'M', 'ASN':'N',
        'PRO':'P', 'GLN':'Q', 'ARG':'R', 'SER':'S',
        'THR':'T', 'VAL':'V', 'TYR':'Y', 'TRP':'W'}
    poc_seq = ''
    i = ''  # locator
    position = []
    protein_seq, order = get_pro_seq(protein_path)
    for line in open(pocket_path):
        if line[0:4] == "ATOM":
            columns = line.split()
            index1 = columns[4]
            index2 = columns[5]
            if len(columns[4]) > 1: # When the residue sequence exceeds 1000, there will be no spaces between the chain and sequence, and manual separation is required
                index1 = columns[4][0]
                index2 = columns[4][1:]
            if index2 != i:
                i = index2
                position.append(order[(index1, index2)])
                poc_seq += aa_codes.get(columns[3], 'X')
            else:
                continue
    return protein_seq, poc_seq, position

# This module is used to obtain the absolute position information of the entire protein sequence and pockets
def get_pro_seq(path):
    aa_codes = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E',
        'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 'LYS': 'K',
        'ILE': 'I', 'LEU': 'L', 'MET': 'M', 'ASN': 'N',
        'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S',
        'THR': 'T', 'VAL': 'V', 'TYR': 'Y', 'TRP': 'W'}
    seq = ''
    order = {}
    i = 0  # locator
    for line in open(path):
        if line[0:4] == "ATOM":
            columns = line.split()
            index1 = columns[4]
            index2 = columns[5]
            if len(columns[4]) > 1: # When the residue sequence exceeds 1000, there will be no spaces between the chain and sequence, and manual separation is required
                index1 = columns[4][0]
                index2 = columns[4][1:]
            if (index1, index2) not in order:
                i = i + 1
                order[(index1, index2)] = i
                seq += aa_codes.get(columns[3], 'X')
            else:
                continue
    return seq, order

pocket_file_path = '../case_study_CB1R/alphafold_DoGsite3_pocket.pdb'
protein_file_path = '../case_study_CB1R/alphafold_protein.pdb'

protein_seq, pocket_seq, position = get_poc_seq(pocket_file_path, protein_file_path)
print(pocket_seq)
print(position)
print(protein_seq)
print(len(protein_seq), len(pocket_seq))

# 读取 CSV 文件
file_path = "./filtered_zinc_SMILES.csv"
df = pd.read_csv(file_path)

# 将 position 列表转换为字符串
position_str = ', '.join(map(str, position))  # 将列表元素转换为字符串并用逗号和空格分隔
position = f'[{position_str}]'  # 添加方括号

# 替换第三列（假设列索引为 2，因为 Python 是从 0 开始计数的）
df.insert(2, 'protein', protein_seq)
df.insert(3, 'pocket', pocket_seq)
df.insert(4, 'position', position)

# 保存到新的 CSV 文件
output_file_path = "../processed_zinc_CB1R.csv"
df.to_csv(output_file_path, index=False)

print(f"文件已成功更新,结果保存在 {output_file_path}")

7. Construct input protein file (in a format of .pkl.gz)

In [ ]:
from Bio.PDB import PDBParser
import gzip
import pickle

def save_dict(mol_dict, output_path):
    with gzip.open(output_path, 'wb') as f:
        pickle.dump(mol_dict, f, protocol=pickle.HIGHEST_PROTOCOL)


AA_MAP = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G',
    'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N',
    'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S', 'THR': 'T', 'VAL': 'V',
    'TRP': 'W', 'TYR': 'Y',  # 20种标准氨基酸
    'SEC': 'U',  # 硒代半胱氨酸 (Selenocysteine)
    'PYL': 'O',  # 吡咯赖氨酸 (Pyrrolysine)
    'HYP': 'B',  # 羟脯氨酸 (Hydroxyproline) 将其从原本的'X'改为'B'，可以避免与未知氨基酸的冲突
    'SEP': 'Z',  # 磷酸丝氨酸 (Phosphoserine)
    'TPO': 'J'   # 磷酸酪氨酸 (Phosphotyrosine)
}

def three_to_one_coords(aa_three_letter):
    # 对于非常见的20种氨基酸，返回 'X'
    return AA_MAP.get(aa_three_letter, 'X')

def three_to_one_seq(aa_three_letter):
    # 如果是常见的20种氨基酸，返回其单字母表示；否则，返回 None
    return AA_MAP.get(aa_three_letter, None)

def extract_protein_coords_and_seq_from_pdb(pdb_file_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file_path)
    protein_sequence = []
    coords = {'N': [], 'CA': [], 'C': [], 'O': []}

    for residue in structure.get_residues():
        if residue.has_id('CA'):
            aa_three_letter = residue.get_resname()
            aa_one_letter_seq = three_to_one_seq(aa_three_letter)
            if aa_one_letter_seq:  # 如果不是 None，即不是非标准氨基酸
                protein_sequence.append(aa_one_letter_seq)
            aa_one_letter_coords = three_to_one_coords(aa_three_letter)
            if aa_one_letter_coords == 'X':  # 如果是非标准氨基酸
                continue
            for atom in residue:
                if atom.name.strip() in ['N', 'CA', 'C', 'O']:
                    coord = list(round(float(x), 3) for x in atom.coord)
                    coords[atom.name.strip()].append(coord)
                    # coords[atom.name.strip()].append(list(atom.coord))
            coords = {k: [list(item) for item in v] for k, v in coords.items()}

    return coords, ''.join(protein_sequence)

pdb_data_dir='../case_study_CB1R/alphafold_DoGsite3_pocket.pdb'
output_pkl_path = f'../pocket_structures.pkl.gz'
coords, protein_sequence = extract_protein_coords_and_seq_from_pdb(pdb_data_dir)
assert (len(coords['CA']) == len(protein_sequence) and len(coords['N']) == len(coords['C']) and
        len(coords['CA']) == len(coords['C']) and len(coords['N']) == len(coords['O']))
protein_dict = {'target': {
    'seq': protein_sequence,
    'coords': coords
}}
save_dict(protein_dict, output_pkl_path)